In [1]:
import models
import datasets
import tensorflow as tf
import tensorflow.keras as keras
from os import path
import json
import keras.backend as K
import numpy as np
from keras_flops import get_flops

# Evaluate

In [2]:
def get_accuracy(model, dataset_name, img_size):
    data_train, data_val, data_test, info = datasets.load_data(dataset_name)
    data_train, data_val, data_test = datasets.preprocess_data(data_train, data_val, data_test, img_size)
    
    result = model.evaluate(data_test)
    return result[1]


def evaluate(backbone_names, dataset_names):
    for dataset_name in dataset_names:
        specs = {}
        for backbone_name in backbone_names:
            specs_one = {}
            model_path = path.join("models", dataset_name, backbone_name)
            model = keras.models.load_model(model_path)
            img_size = models.get_image_size(backbone_name)
            
            acc = get_accuracy(model, dataset_name, img_size)
            specs_one["accuracy"] = acc
            
            specs_one["parameters"] = model.count_params()
            
            # iz nekog razloga vraca fuplo FLOPs pa podijeli s 2
            specs_one["flops"] = get_flops(model, batch_size=1) / 2
            
            specs[backbone_name] = specs_one
            
        save_path = path.join("best_models_log", dataset_name, "evaluate.txt")
        with open(save_path, "w") as file:
            file.write(json.dumps(specs))

In [3]:
backbone_names = ["effnetB0", "effnetB1", "effnetB2", "effnetB3",
                  "densenet121", "densenet169",
                  "resnet101v2", "resnet152v2",
                  "mobilenetv2"]

In [4]:
dataset_names = ["oxford_iiit_pet"]
evaluate(backbone_names, dataset_names)

115/115 [==============================] - 18s 88ms/step - loss: 0.3335 - accuracy: 0.9041
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


115/115 [==============================] - 7s 53ms/step - loss: 0.3835 - accuracy: 0.9005


In [5]:
dataset_names = ["oxford_flowers102"]
evaluate(backbone_names, dataset_names)

32/32 [==============================] - 3s 62ms/step - loss: 0.2860 - accuracy: 0.9167


In [6]:
dataset_names = ["uc_merced"]
evaluate(backbone_names, dataset_names)

10/10 [==============================] - 2s 88ms/step - loss: 0.1891 - accuracy: 0.9397
